# Used-Cars

## Programming for Data Science Course from University of Science
## Final project



### Team members
<center>

|Thành viên           |MSSV     |
|:-------------------|:-------|
|Vũ Minh Chiến        |22127045 |
|Quách Trần Quán Vinh |22127460 |

</center>

### **1. Collecting data**

What subject is your data about?
- Craigslist is the world's largest collection of used vehicles for sale, this dataset includes part of used vehicle entry within the United States on Craigslist.

What is the source of your data?
- The source of this data set can be found in [kaggle](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data/data). For this project, we have reduce the origin data set, which contains 426880 rows, down to 150000 rows (the reason for this will be mentioned in reflection part).

Do authors of this data allow you to use like this?
- Yes, this data set have `CC0: Public Domain` license (you can find more informaitons about the license [here](https://creativecommons.org/publicdomain/zero/1.0/)).

How did authors collect data?
- The arthor collect data by a web application built with Flask which scrapes used vehicle listings weekly from all regional Craigslist sites in the United States (you can find more informaitons on scraping process [here](https://github.com/AustinReese/UsedVehicleSearch)).

Import necessary libraries

In [6]:
import zipfile
import pandas as pd

Read the dataset

In [7]:
df = pd.read_csv('vehicles.zip', delimiter=',',compression='zip')

### **2. Exploring data**

In [8]:
# display dataset
df.sample(5)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
96142,7307101383,https://westernmass.craigslist.org/ctd/d/india...,western massachusetts,https://westernmass.craigslist.org,1000,2017.0,toyota,camry,NaN,4 cylinders,...,NaN,sedan,grey,https://images.craigslist.org/00V0V_aSoHChDq0D...,2017 *Toyota* *Camry* SE Automatic SedanCall o...,NaN,ma,42.145814,-72.490575,2021-04-15T08:00:18-0400
51933,7314571406,https://medford.craigslist.org/ctd/d/medford-2...,medford-ashland,https://medford.craigslist.org,12995,2005.0,ford,f350 xlt crew cab 4x4,excellent,8 cylinders,...,full-size,truck,red,https://images.craigslist.org/01111_g4tjBOMwsR...,"2005 FORD F350 XLT , 8 FOOT BED , 4X4 , CREW C...",NaN,or,42.331105,-122.872603,2021-04-29T15:42:01-0700
79176,7316530127,https://santamaria.craigslist.org/ctd/d/los-an...,santa maria,https://santamaria.craigslist.org,34500,2019.0,ram,promaster 1500,excellent,6 cylinders,...,NaN,van,white,https://images.craigslist.org/00101_gaxpNibInz...,"LA TRUCK GUY 1211 WEST EL SEGUNDO BLVD, GARDEN...",NaN,ca,33.916305,-118.296520,2021-05-03T17:36:22-0700
34765,7313922215,https://harrisburg.craigslist.org/ctd/d/harris...,harrisburg,https://harrisburg.craigslist.org,36990,2015.0,chevrolet,tahoe lt sport utility,good,8 cylinders,...,NaN,other,black,https://images.craigslist.org/00U0U_cvoZgipqRa...,Carvana is the safer way to buy a car During t...,NaN,pa,40.270000,-76.880000,2021-04-28T13:40:56-0400
34068,7310826972,https://greenville.craigslist.org/ctd/d/piedmo...,greenville / upstate,https://greenville.craigslist.org,6939,2009.0,mini,cooper s,good,4 cylinders,...,compact,hatchback,yellow,https://images.craigslist.org/00C0C_8stHdsF4tB...,This 2009 Mini Cooper S 1.6L I4 Turbocharger ...,NaN,sc,34.700405,-82.471923,2021-04-22T10:45:54-0400
